# Determine derivative of Jacobian from angular velocity to exponential rates

Peter Corke 2021, updated 1/23

SymPy code to determine the time derivative of the mapping from angular velocity to exponential coordinate rates.

In [ ]:
from sympy import *

A rotation matrix can be expressed in terms of exponential coordinates (also called the Euler vector)

$
\mathbf{R} = e^{[\varphi]_\times} 
$
where $\mathbf{R} \in SO(3)$ and $\varphi \in \mathbb{R}^3$.

The mapping from angular velocity $\omega$ to exponential coordinate rates $\dot{\varphi}$ is

$
\dot{\varphi} = \mathbf{A} \omega
$

where $\mathbf{A}$ is given by (2.107) of [Robot Dynamics Lecture Notes, Robotic Systems Lab, ETH Zurich, 2017](https://ethz.ch/content/dam/ethz/special-interest/mavt/robotics-n-intelligent-systems/rsl-dam/documents/RobotDynamics2017/RD_HS2017script.pdf)


$
\mathbf{A} = \mathbf{1}_{3 \times 3} - \frac{1}{2} [\varphi]_\times + [\varphi]^2_\times \frac{1}{\theta^2} \left( 1 - \frac{\theta}{2} \frac{\sin \theta}{1 - \cos \theta} \right),
$
where $\theta = \| \varphi \|$

We simplify the equation as

$
\mathbf{A} = \mathbf{1}_{3 \times 3} - \frac{1}{2} [\varphi]_\times + [\varphi]^2_\times \Theta
$

where
$
\Theta = \frac{1}{\theta^2} \left( 1 - \frac{\theta}{2} \frac{\sin \theta}{1 - \cos \theta} \right)
$

We can find the derivative using the chain rule

$
\dot{\mathbf{A}} = - \frac{1}{2} [\dot{\varphi}]_\times +  \left( [\varphi]_\times [\dot{\varphi}]_\times + [\dot{\varphi}]_\times[\varphi]_\times  \right) \Theta + [\varphi]^2_\times \dot{\Theta}
$

noting that the derivative of a matrix squared is

$
\frac{d}{dt} (\mathbf{M}^2) = (\frac{d}{dt} \mathbf{M}) \mathbf{M} + \mathbf{M} (\frac{d}{dt} \mathbf{M})
$

We start by defining some symbols

In [ ]:
theta, theta_dot, t = symbols('theta theta_dot t', real=True)

We start by finding an expression for $\Theta$ which depends on $\theta(t)$

In [ ]:
theta_t = Function(theta)(t)
theta_t

In [ ]:
Theta = 1 / theta_t ** 2 * (1 - theta_t / 2 * sin(theta_t) / (1 - cos(theta_t)))
Theta

and now determine the derivative

In [ ]:
T_dot = Theta.diff(t)
T_dot

which is a somewhat complex expression that depends on $\theta(t)$ and $\dot{\theta}(t)$.

We will remove the time dependency and generate code

In [ ]:
T_dot = T_dot.subs([(theta_t.diff(t), theta_dot), (theta_t, theta)])
T_dot

In [ ]:
pycode(T_dot)

In order to evaluate the line above we need an expression for $\theta$ and $\dot{\theta}$.  $\theta$ is the norm of $\varphi$ whose elements are functions of time

In [ ]:
phi_names = ('varphi_0', 'varphi_1', 'varphi_2')
phi = []  # names of angles, eg. theta
phi_t = []  # angles as function of time, eg. theta(t)
phi_d = []  # derivative of above, eg. d theta(t) / dt
phi_n = []  # symbol to represent above, eg. theta_dot
for i in phi_names:
    phi.append(symbols(i, real=True))
    phi_t.append(Function(phi[-1])(t))
    phi_d.append(phi_t[-1].diff(t))
    phi_n.append(i + '_dot')

Compute the norm

In [ ]:
theta = Matrix(phi_t).norm()
theta

and find its derivative

In [ ]:
theta_dot = theta.diff(t)
theta_dot

and now remove the time dependenices

In [ ]:
theta_dot = theta_dot.subs(a for a in zip(phi_d, phi_n))
theta_dot = theta_dot.subs(a for a in zip(phi_t, phi))
theta_dot

which is simply the dot product over the norm.

In [ ]:
A, t = symbols('A t', real=True)
A_t = Function(A)(t)
d = diff(exp(A_t), t)
print(d)